In [1]:
%matplotlib inline 
import requests
import pprint # for pretty printing
import os # listing and managing file patho
import zipfile # for zip and unzip utilities
import pandas # for data analysis
import csv#导入numpy的库函数
import numpy as np
import matplotlib.pyplot as plt # for plotting
from textblob import TextBlob
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer # for converting documents in word count
from cvxopt import matrix, solvers, spmatrix, sparse
from sklearn import svm
from array import array

In [2]:
with  open('./data/SMSSpamCollection', 'r') as f:
    sms_messages = f.readlines()
sms_messages = [m.rstrip() for m in sms_messages]
messages = pandas.read_csv('./data/SMSSpamCollection', sep='\t', quoting=csv.QUOTE_NONE,
                           names=["label", "message"])
messages['length'] = messages['message'].map(lambda text: len(text))
def split_into_tokens(message):
    return TextBlob(message).words 
messages.message.head().apply(split_into_tokens)
def split_into_lemmas(message):
    words = TextBlob(message).words.lower()
    # for each word, take its "base form" = lemma 
    return [word.lemma for word in words]

# see how head portion changes
messages.message.head().apply(split_into_lemmas)
bow_transformer = CountVectorizer(analyzer=split_into_lemmas).fit(messages['message'])
messages_bow = bow_transformer.transform(messages['message'])#messages_bow是数组化的文本了

In [3]:
np.random.seed(1)
N= 5574# number of sms message
train_examples = int(5574*80/100)
test_examples = N- train_examples
random_indices = np.random.permutation(range(N))
train_indices = random_indices[:train_examples]
assert len(train_indices) == train_examples, "slicing issue"
test_indices = random_indices[train_examples: N+1]
assert len(test_indices) == test_examples, "slicing issue"
X=messages_bow[train_indices]
K=messages_bow[train_indices]
_Y=messages.label[train_indices]
Y=np.zeros((_Y.shape[0]))
for i,j in zip(range(_Y.shape[0]),_Y):
    if j == 'spam':
        Y[i]=1
    else:
        Y[i]=-1
print(Y)   

[-1. -1. -1. ..., -1.  1. -1.]


In [4]:
n=X.shape[0]
for i in range(n):
    K[i]=Y[i]*K[i]    
K=np.dot(K,K.T)

In [5]:
def scipy_sparse_to_spmatrix(A):
    coo = A.tocoo()
    SP = spmatrix(coo.data.tolist(), coo.row.tolist(), coo.col.tolist(), size=A.shape)
    return SP

In [6]:
P=scipy_sparse_to_spmatrix(K)
q=matrix(-np.ones((n, 1)))
G=matrix(-np.eye(n))
h=matrix(np.zeros(n))
b=matrix(np.zeros(1))
A=matrix(Y.reshape(1,4459))
solution=solvers.qp(P, q, G, h, A, b)
arfa=solution['x']
arfa=np.array(arfa)

     pcost       dcost       gap    pres   dres
 0: -4.8615e+01 -1.5859e+02  1e+04  8e+01  2e+00
 1: -3.0513e+01 -1.5212e+02  3e+03  2e+01  4e-01
 2: -1.9294e+01 -8.5681e+01  6e+02  4e+00  9e-02
 3: -1.3239e+01 -5.9550e+01  2e+02  1e+00  3e-02
 4: -1.2279e+01 -4.2129e+01  6e+01  2e-01  5e-03
 5: -1.6173e+01 -3.0174e+01  2e+01  5e-02  1e-03
 6: -1.8538e+01 -2.6146e+01  8e+00  2e-03  4e-05
 7: -2.0666e+01 -2.4086e+01  3e+00  5e-04  1e-05
 8: -2.1728e+01 -2.3318e+01  2e+00  7e-15  1e-14
 9: -2.2506e+01 -2.2892e+01  4e-01  5e-15  1e-14
10: -2.2728e+01 -2.2804e+01  8e-02  2e-15  1e-14
11: -2.2787e+01 -2.2791e+01  4e-03  8e-16  1e-14
12: -2.2790e+01 -2.2790e+01  7e-05  4e-15  1e-14
13: -2.2790e+01 -2.2790e+01  2e-06  2e-15  1e-14
Optimal solution found.


In [7]:
w=np.zeros((1,8859))
for i in range(n):
    w+=arfa[i]*Y[i]*X[i]
print(w)    

[[ -1.11554251e-09  -2.89385243e-10   0.00000000e+00 ...,  -3.81481528e-10
   -2.81548265e-07  -2.81548265e-07]]


In [8]:
w0=Y.reshape(4459,1)-X.dot(w.T)
_w0=np.min(w0)
bia_opt=0.6
print(_w0)

-5.41960853858


In [9]:
norm=np.linalg.norm(w)
w,_w0=w/norm,bia_opt+_w0/norm

In [10]:
testdata=messages_bow[test_indices]
result=[]
result=testdata.dot(w.T)+_w0

In [11]:
prediction=['a']*result.shape[0]
for i in range (result.shape[0]):
    if result[i]>0:
        prediction[i]='spam'
    else:
        prediction[i]='ham'

In [12]:
print ('accuracy', accuracy_score(messages.label[test_indices],prediction))

accuracy 0.983856502242


In [13]:
clf = svm.LinearSVC()  
clf.fit(X,Y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [14]:
result_skl = clf.predict(testdata)
prediction_skl=['a']*result_skl.shape[0]
for i in range (result_skl.shape[0]):
    if result_skl[i]>0:
        prediction_skl[i]='spam'
    else:
        prediction_skl[i]='ham'       

In [15]:
print ('accuracy', accuracy_score(messages.label[test_indices],prediction_skl))

accuracy 0.983856502242
